# Init

In [9]:
import os
os.chdir('..')
os.getcwd()

'/Users/wliao0504/code/clif/pyCLIF'

## Reload

In [10]:
import pandas as pd
from pathlib import Path
import duckdb

# Import individual table classes
from clifpy.tables.vitals import Vitals
from clifpy.tables.labs import Labs
from clifpy.tables.medication_admin_continuous import MedicationAdminContinuous

from importlib import reload

import yaml

with open('config/config.yaml', 'r') as file:
    config = yaml.safe_load(file)
    
from clifpy.utils import sofa
reload(sofa)

<module 'clifpy.utils.sofa' from '/Users/wliao0504/code/clif/pyCLIF/clifpy/utils/sofa.py'>

In [ ]:
# Initialize the tables
vitals = Vitals.from_file(
    data_directory=config['tables_path'],
    filetype='parquet', 
    timezone='UTC'
)

vitals_df = vitals.df

# Initialize the tables
labs = Labs.from_file(
    data_directory=config['tables_path'],
    filetype='parquet', 
    timezone='UTC'
)

labs_df = labs.df

# Initialize the tables
mac = MedicationAdminContinuous.from_file(
    data_directory=config['tables_path'],
    filetype='parquet', 
    timezone='UTC'
)

mac_df = mac.df

Loading clif_vitals.parquet
Data loaded successfully from clif_vitals.parquet
recorded_dttm: null count before conversion= 0
recorded_dttm: Your timezone is UTC, Converting to your site timezone (UTC).
recorded_dttm: null count after conversion= 0
Validation completed with 1 error(s). See `errors` attribute.
Loading clif_labs.parquet
Data loaded successfully from clif_labs.parquet
lab_order_dttm: null count before conversion= 43419
lab_order_dttm: Your timezone is UTC, Converting to your site timezone (UTC).
lab_order_dttm: null count after conversion= 43419
lab_collect_dttm: null count before conversion= 0
lab_collect_dttm: Your timezone is UTC, Converting to your site timezone (UTC).
lab_collect_dttm: null count after conversion= 0
lab_result_dttm: null count before conversion= 0
lab_result_dttm: Your timezone is UTC, Converting to your site timezone (UTC).
lab_result_dttm: null count after conversion= 0
Validation completed with 24 error(s). See `errors` attribute.
Loading clif_medi

In [ ]:
# Mock DataFrame creation for ids_w_dttm
cohort_df = pd.DataFrame({
    'hospitalization_id': ['23559586', '20626031'],
    'start_time': pd.to_datetime(['2132-12-15 14:29:00+00:00', '2132-12-14 21:00:00+00:00']),
    'end_time': pd.to_datetime(['2137-08-25 14:00:00+00:00', '2137-09-02 09:00:00+00:00'])
})

In [ ]:
query_dict = {'weight_kg': ['latest', 'max'], 'spo2': ['max', 'min']}

# pivoted = sofa.lookup_extremal_vital_values(ids_w_dttm, query_dict, vitals_df, )

### Generic

In [ ]:
df = sofa.lookup_extremal_values_in_long_table(cohort_df, query_dict, vitals_df, "vitals")

df

,hospitalization_id,start_dttm,end_dttm,spo2_max,spo2_min,weight_kg_latest,weight_kg_max
0,20626031,2132-12-14 15:00:00-06:00,2137-09-02 03:00:00-06:00,100.0,91.0,95.1,95.1
1,23559586,2132-12-15 08:29:00-06:00,2137-08-25 08:00:00-06:00,100.0,87.0,123.6,124.5


In [ ]:
query_dict = {"propofol": ["latest", "max"], "fentanyl": ["latest", "min"]}

sofa.lookup_extremal_values_in_long_table(cohort_df, query_dict, "medication_admin_continuous")

,hospitalization_id,start_dttm,end_dttm,fentanyl_latest,fentanyl_min,propofol_latest,propofol_max,propofol_min
0,20626031,2132-12-14 15:00:00-06:00,2137-09-02 03:00:00-06:00,NaN,NaN,0.0,50.229695,0.0
1,23559586,2132-12-15 08:29:00-06:00,2137-08-25 08:00:00-06:00,0.0,0.0,0.0,30.026425,0.0


In [ ]:
query_dict = {
    'hemoglobin': ['latest', 'max'], 
    'platelet_count': ['max', 'min'],
    'bicarbonate': ['latest', 'min']
    }

sofa.lookup_extremal_values_in_long_table(cohort_df, query_dict, labs_df, "labs")

,hospitalization_id,start_dttm,end_dttm,bicarbonate_latest,bicarbonate_min,hemoglobin_latest,hemoglobin_max,platelet_count_latest,platelet_count_max,platelet_count_min
0,20626031,2132-12-14 15:00:00-06:00,2137-09-02 03:00:00-06:00,27.0,22.0,7.9,9.8,NaN,375.0,192.0
1,23559586,2132-12-15 08:29:00-06:00,2137-08-25 08:00:00-06:00,21.0,18.0,7.4,9.3,61.0,115.0,61.0


## New

In [9]:
melted = qualified.melt(
    id_vars=['hospitalization_id', 'start_dttm', 'end_dttm', 'vital_category', 'vital_value'],
    value_vars=['rn_max', 'rn_latest', 'rn_min'],
    var_name='extreme',
    value_name='rn'
).query('rn == 1').drop(columns=['rn'])

NameError: name 'qualified' is not defined

In [ ]:
# Create column names like 'spo2_max', 'weight_kg_latest'
melted['column_name'] = melted['vital_category'] + '_' + melted['extreme'].str.replace('rn_', '')

# Pivot to get the desired column structure
pivoted = melted.pivot_table(
    index=['hospitalization_id', 'start_dttm', 'end_dttm'],
    columns='column_name',
    values='vital_value'
).reset_index()

In [ ]:
pivoted

,hospitalization_id,start_dttm,end_dttm,spo2_max,spo2_min,weight_kg_latest,weight_kg_max
0,20626031,2132-12-14 15:00:00-06:00,2137-09-02 03:00:00-06:00,100.0,91.0,95.1,95.1
1,23559586,2132-12-15 08:29:00-06:00,2137-08-25 08:00:00-06:00,100.0,87.0,123.6,124.5


# Using Wide

In [11]:
from clifpy.utils import create_wide_dataset
from clifpy.clif_orchestrator import ClifOrchestrator

In [24]:
co = ClifOrchestrator(
    data_directory='/Users/wliao0504/code/clif/pyCLIF/clifpy/data/clif_demo',
    filetype='parquet',
    timezone='UTC',
    output_directory= None
)

co.load_table('vitals')
co.load_table('labs')
co.load_table('patient_assessments')
co.load_table('medication_admin_continuous')
co.load_table('respiratory_support')

ClifOrchestrator initialized.
Loading clif_vitals.parquet
Data loaded successfully from clif_vitals.parquet
recorded_dttm: Already in your timezone (UTC), no conversion needed.
Loading clif_labs.parquet
Data loaded successfully from clif_labs.parquet
lab_order_dttm: Already in your timezone (UTC), no conversion needed.
lab_collect_dttm: Already in your timezone (UTC), no conversion needed.
lab_result_dttm: Already in your timezone (UTC), no conversion needed.
Loading clif_patient_assessments.parquet
Data loaded successfully from clif_patient_assessments.parquet
recorded_dttm: Already in your timezone (UTC), no conversion needed.
Loading clif_medication_admin_continuous.parquet
Data loaded successfully from clif_medication_admin_continuous.parquet
admin_dttm: Already in your timezone (UTC), no conversion needed.
Loading clif_respiratory_support.parquet
Data loaded successfully from clif_respiratory_support.parquet
recorded_dttm: Already in your timezone (UTC), no conversion needed.


In [25]:
cohort_df = pd.DataFrame({
    'hospitalization_id': ['23559586', '20626031'],
    'start_time': pd.to_datetime(['2137-01-01 14:29:00+00:00', '2132-12-14 21:00:00+00:00']),
    'end_time': pd.to_datetime(['2137-08-25 14:00:00+00:00', '2132-12-15 09:00:00+00:00'])
})

required_sofa_categories_by_table = {
    'labs': ['creatinine','platelet_count','po2_arterial','bilirubin_total'],
    'vitals': ['map','spo2', 'weight_kg'],
    'patient_assessments': ['gcs_total'],
    "medication_admin_continuous": [
        "norepinephrine","epinephrine","phenylephrine","vasopressin", "dopamine","angiotensin",
        "dobutamine","milrinone"
        ],
    'respiratory_support': [
        'device_category','device_name','mode_name','mode_category','peep_set','fio2_set','lpm_set',
        'resp_rate_set','tracheostomy', 'resp_rate_obs','tidal_volume_set'
    ] 
}

wide_df = co.create_wide_dataset(
    tables_to_load=['vitals', 'labs', 'patient_assessments', 'medication_admin_continuous'],
    category_filters=required_sofa_categories_by_table,
    # sample=True,  # Use 20 random hospitalizations
    cohort_df=cohort_df,
    batch_size=-1
)

Loading patient table...
Loading clif_patient.parquet
Data loaded successfully from clif_patient.parquet
death_dttm: Already in your timezone (UTC), no conversion needed.
Loading hospitalization table...
Loading clif_hospitalization.parquet
Data loaded successfully from clif_hospitalization.parquet
admission_dttm: Already in your timezone (UTC), no conversion needed.
discharge_dttm: Already in your timezone (UTC), no conversion needed.
Loading adt table...
Loading clif_adt.parquet
Data loaded successfully from clif_adt.parquet
in_dttm: Already in your timezone (UTC), no conversion needed.
out_dttm: Already in your timezone (UTC), no conversion needed.
Starting wide dataset creation...
Using cohort_df with time windows for 2 hospitalizations
Using 2 hospitalization IDs from cohort_df

Loading and filtering base tables...
Base tables filtered - Hospitalization: 2, Patient: 100, ADT: 11

=== Processing Tables ===
Base cohort created with 2 records

Processing labs...
Loaded 931 records fr

In [27]:
required_sofa_categories = [item for sublist in required_sofa_categories_by_table.values() for item in sublist]

In [28]:
q = f"""
FROM wide_df
SELECT hospitalization_id
    -- , MAX(COLUMNS({required_sofa_categories})) AS 'max_\\0'
    , any_value(COLUMNS({required_sofa_categories}) ORDER BY event_time DESC) -- AS 'latest_\\0'
GROUP BY hospitalization_id
"""
extremals_by_id = duckdb.sql(q).df()

In [ ]:
q = f"""
FROM extremals_by_id
SELECT hospitalization_id
    , sofa_cv_97: CASE WHEN dopamine > 15 OR epinephrine > 0.1 OR norepinephrine > 0.1 THEN 4
        WHEN dopamine > 5 OR epinephrine <= 0.1 OR norepinephrine <= 0.1 THEN 3
        WHEN dopamine <= 5 OR dobutamine > 0 THEN 2
        WHEN map < 70 THEN 1
        WHEN map >= 70 THEN 0 END
    , sofa_coag: CASE WHEN platelet_count < 20 THEN 4
        WHEN platelet_count < 50 THEN 3
        WHEN platelet_count < 100 THEN 2
        WHEN platelet_count < 150 THEN 1
        WHEN platelet_count >= 150 THEN 0 END 
    , sofa_liver: CASE WHEN bilirubin_total >= 12 THEN 4
        WHEN bilirubin_total < 12 AND bilirubin_total >= 6 THEN 3
        WHEN bilirubin_total < 6 AND bilirubin_total >= 2 THEN 2
        WHEN bilirubin_total < 2 AND bilirubin_total >= 1.2 THEN 1
        WHEN bilirubin_total < 1.2 THEN 0 END
    , sofa_cns: CASE WHEN gcs_total < 6 THEN 4
        WHEN gcs_total >= 6 AND gcs_total <= 9 THEN 3
        WHEN gcs_total >= 10 AND gcs_total <= 12 THEN 2
        WHEN gcs_total >= 13 AND gcs_total <= 14 THEN 1
        WHEN gcs_total == 15 THEN 0 END
    , sofa_renal: CASE WHEN creatinine >= 5 THEN 4
        WHEN creatinine < 5 AND creatinine >= 3.5 THEN 3
        WHEN creatinine < 3.5 AND creatinine >= 2 THEN 2
        WHEN creatinine < 2 AND creatinine >= 1.2 THEN 1
        WHEN creatinine < 1.2 THEN 0 END
    , sofa_total: sofa_cv_97 + sofa_coag + sofa_liver + sofa_renal + sofa_cns
"""
df = duckdb.sql(q).df()